# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# МНОГИЕ ФУНКЦИИ ПОДПРАВЛЕНЫ В ДЗ4 (лучше туда смотреть)

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [1]:
from gensim.models import Word2Vec, KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import judicial_splitter
from pymystem3 import Mystem
mystem = Mystem()
import numpy as np
from tqdm import tqdm_notebook as tqdm
import os
import pandas as pd
from gensim.test.utils import get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import operator
import re
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
import math

In [2]:
# если модель без тэгов
model_without_pos = Word2Vec.load('/Users/irene/Downloads/IR/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [17]:
# если модель с POS-тэггингом
# model_with_pos = KeyedVectors.load_word2vec_format('/Users/irene/Downloads/IR/tayga_1_2.vec', binary=False)

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [4]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [5]:
def get_w2v_vectors_paragraph(paragraph, model):
    """Получает вектор для параграфа"""
    lemmas_paragraph = preprocessing(paragraph)
    if len(lemmas_paragraph) == 0:
        return np.zeros(300)
    else:
        vector_paragraph = []
        for lemma in lemmas_paragraph:
            try:
                vector = model.wv[lemma]
            except:
                vector = np.zeros(300)
            vector_paragraph.append(vector)
        vec = np.array(vector_paragraph).sum(axis=0) / len(vector_paragraph)
        return vec.tolist()

In [6]:
def get_w2v_vectors_text(text, model, len_par=4):
    """Получает массив векторов параграфов"""
    paragraphs = judicial_splitter.splitter(text, len_par)
    return [(paragraph, get_w2v_vectors_paragraph(paragraph, model)) for paragraph in paragraphs]

In [7]:
def save_w2v_base(files_list, model):
    """Индексирует всю базу для поиска через word2vec"""
    df = {'Name of file': [], 'Text of paragraph': [], 'Vector of paragraph (w2v)': []}
    for file in tqdm(files_list):
        with open(file, 'r') as f:  
            text = f.read() 
            v_paragraphs = get_w2v_vectors_text(text, model)
            for v_p in v_paragraphs:
                df['Name of file'].append(file)
                df['Text of paragraph'].append(v_p[0])
                df['Vector of paragraph (w2v)'].append(v_p[1])
    return df

In [ ]:
main_dir = '/Users/irene/Downloads/IR/article'
files_list = []
for root, dirs, files in os.walk(main_dir):
    for name in files:
        if not '.DS_Store' in name:
            files_list.append(os.path.join(root, name))

In [ ]:
df = save_w2v_base(files_list, model_without_pos)

In [ ]:
df = pd.DataFrame(data=df)

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [ ]:
def train_doc2vec(data):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, 
                min_alpha=0.025, epochs=100, workers=4, dm=1)

    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [ ]:
fname = get_tmpfile("model_doc2vec_5")
model_doc2vec = train_doc2vec(df['Text of paragraph'])
model_doc2vec.save(fname)

In [ ]:
model_doc2vec = Doc2Vec.load(fname)

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [ ]:
def get_d2v_vectors(paragraph, model_doc2vec):
    """Получает вектор параграфа"""
    lemmas_paragraph = preprocessing(paragraph, del_stopwords=False)
    model_doc2vec.random.seed(100)
    vector = model_doc2vec.infer_vector(lemmas_paragraph)
    return vector.tolist()

def save_d2v_base(paragraphs, model_doc2vec):
    """Индексирует всю базу для поиска через doc2vec"""
    vectors_d2v = []
    for par in tqdm(paragraphs):
        vectors_d2v.append(get_d2v_vectors(par, model_doc2vec))
    return vectors_d2v

In [ ]:
df['Vector of paragraph (d2v)'] = save_d2v_base(df['Text of paragraph'], model_doc2vec)

In [ ]:
df.to_csv('df_5.csv')

In [9]:
df = pd.read_csv('df_5.csv')

In [ ]:
df = pd.read_csv('df_5.csv')

In [10]:
df.head()

,Unnamed: 0,Name of file,Text of paragraph,Vector of paragraph (w2v)
0,0,/Users/irene/Downloads/IR/article/1.txt,Судья Высшего Арбитражного Суда Российской Ф...,"[0.025288590806380136, 0.010929809320750314, 0..."
1,1,/Users/irene/Downloads/IR/article/1.txt,Возвращение к проверке в порядке надзора судеб...,"[0.01896677758187464, 0.011468302054260538, -0..."
2,2,/Users/irene/Downloads/IR/article/10.txt,Судья Высшего Арбитражного Суда Российской Ф...,"[0.02874657063457732, 0.01592562360706452, 0.0..."
3,3,/Users/irene/Downloads/IR/article/100.txt,Высший Арбитражный Суд Российской Федерации ...,"[0.03278444460856037, 0.012549695896825964, -0..."
4,4,/Users/irene/Downloads/IR/article/100.txt,Председательствующий судья Е.Н.ЗАРУБИНА Судья ...,"[0.01786809911330541, 0.03585122898221016, 0.0..."


# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [11]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [12]:
def res_v(vectors, names_doc, v_quary):
    res = []
    for i, vector in enumerate(vectors):
        cos_sim = similarity(v_quary, vector)
        res.append((names_doc[i], cos_sim))
    res.sort(key=operator.itemgetter(1), reverse=True)
    return res

In [13]:
def search_w2v(quary, model, vectors_w2v, names_doc):
    v_quary = get_w2v_vectors_paragraph(quary, model)
    res = res_v(vectors_w2v, names_doc, v_quary)
    return res

def search_d2v(quary, model, vectors_d2v, names_doc):
    v_quary = get_d2v_vectors(paragraph, model)
    res = res_v(vectors_d2v, names_doc, v_quary)
    return res

In [16]:
#res = search_w2v('Судья Высшего Арбитражного Суда Российской Федерации Ю.Ю.Горячева, рассмотрев заявление Комитета по управлению городским имуществом Санкт-Петербурга', model_without_pos, df['Vector of paragraph (w2v)'][0:40000], df['Name of file'][0:40000])

In [ ]:
#res[0:20]

In [ ]:
#df['Name of file'][35000]

In [ ]:
#df['Text of paragraph'][35000]

# Обратный индекс

In [ ]:
answers = []
for f in files_list:
    with open(file, 'r') as f:  
        text = f.read()
        answers.append(text)

In [ ]:
lemmatized_texts = []
for each_f in tqdm(answers):
    lemmatized = ' '.join([x for x in preprocessing(each_f) if x != ' '])
    lemmatized_texts.append(lemmatized)

In [ ]:
vec = CountVectorizer()
X = vec.fit_transform(lemmatized_texts)
df_index = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
words = list(vec.get_feature_names())

In [ ]:
df_index.head()

In [ ]:
def inverted_index(df) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    files = []
    for word in df:
        sub = []
        docs = np.where(df[word] > 0)[0]
        for f in docs:
            dl = len(lemmatized_texts[f].split())
            fr = round(df[word][f]/dl, 4)
            sub.append([f, dl, fr])
        files.append(sub)
    index = pd.DataFrame(data={'Слово': words, 'Информация': files})
    return index

In [ ]:
index = inverted_index(df_index)

In [ ]:
from math import log

k1 = 2.0
b = 0.75
avgdl = round(sum([len(q.split(' ')) for q in lemmatized_texts])/len(lemmatized_texts))#средняя длина док-ов в коллекции
N = len(lemmatized_texts)

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = math.log((N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

In [ ]:
def compute_sim(lemma, inverted_index) -> float:
    """
    Compute similarity score between word in search query and all document  from collection
    :return: score
    """
    #print(list(inverted_index.loc[inverted_index['Слово'] == lemma]['Информация']))
    doc_list = list(inverted_index.loc[inverted_index['Слово'] == lemma]['Информация'])[0]
    relevance_dict = {}
    for doc in doc_list:
        relevance_dict[doc[0]] = score_BM25(doc[2], doc[1], avgdl, k1, b, N, len(doc_list))
    return relevance_dict

In [ ]:
def get_search_result(query, top=5) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    query = [que for que in preprocessing(query) if que in words]
    #print(query)
    res = {}
    for word in query:
        relevance_dict = compute_sim(word, index)
        res = {k: res.get(k, 0) + relevance_dict.get(k, 0) for k in set(res) | set(relevance_dict)}
    return sorted(res.items(), key=operator.itemgetter(1), reverse=True)[0:top]

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [ ]:
def search(search_method):
    if search_method == 'inverted_index':
        search_result = get_search_result(query)
    elif search_method == 'word2vec':
        search_result = search_w2v()
    elif search_method == 'doc2vec':
        search_result = search_d2v()
    else:
        raise TypeError('unsupported search method')